In [ ]:
!pip install -q facenet_pytorch
!pip install -q mmcv

     |████████████████████████████████| 1.9 MB 13.5 MB/s 
     |████████████████████████████████| 438 kB 13.5 MB/s 
     |████████████████████████████████| 190 kB 47.6 MB/s 


# Face tracking pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and tracking on an image dataset using MTCNN.

In [ ]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

#### Determine if an nvidia GPU is available

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [ ]:
mtcnn = MTCNN(keep_all=True, device=device)

#### Get a sample video

We begin by loading a video with some faces in it. The `mmcv` PyPI package by mmlabs is used to read the video frames (it can be installed with `pip install mmcv`). Frames are then converted to PIL images.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# video = mmcv.VideoReader('/content/drive/MyDrive/video.mp4')
# frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

# display.Video('/content/drive/MyDrive/video.mp4', width=640)

In [ ]:
# mmcv.cut_video('/content/drive/MyDrive/video.mp4', '/content/drive/MyDrive/clip.mp4', start=60, end=120, vcodec='h264')
video = mmcv.VideoReader('/content/drive/MyDrive/clip.mp4')

#### Run video through MTCNN

We iterate through each frame, detect faces, and draw their bounding boxes on the video frames.

In [ ]:
import numpy as np


POINTS = np.array((
    (250, 1440),
    (515, 888), 
    (850, 450),
    (1350, 450),
    (1700, 600),
    (2560, 600),
    (2560, 1440),
    (250, 1440)
))


def is_inside(point: np.ndarray, polygon: np.ndarray) -> bool:
    in_polygon = False
    x, y = point
    for i in range(polygon.shape[0]):
        curr_x, curr_y = polygon[i]
        prev_x, prev_y = polygon[i - 1]
        if ((curr_y <= y and y < prev_y) or (prev_y <= y and y < curr_y)) and \
           (x > (prev_x - curr_x) * (y - curr_y) / (prev_y - curr_y) + curr_x):
                in_polygon = not in_polygon
    return in_polygon

In [ ]:
frames_tracked = []
for i, frame in enumerate(video, start=1):

    frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    print(f'\rTracking frame: {i}', end='')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    # Draw faces
    frame_draw = frame.copy()
    drawer = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            # up_left, down_right = box[:2], box[2:]
            # if is_inside(up_left, points) and is_inside(down_right, points):
            #     draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
            drawer.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')

Tracking frame: 1500
Done


#### Display detections

#### Save tracked video

In [ ]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('/content/drive/MyDrive/video_tracked.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()